In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import math
from comtrade import Comtrade
import plotly.graph_objects as go
import plotly.express as px
plt.rcParams['savefig.directory'] = os.getcwd()

# change this to match the exported COMTRADE file
root = '../data/playback'

# 1..3 = Va..Vc
# 4..6 = Ia..Ic
# 7 = Vrms
# 8 = P
# 9 = Q
# load all the analog channels into a dictionary of numpy arrays
rec = Comtrade ()
rec.load (root + '.cfg', root + '.dat')
t = np.array(rec.time)
channels = {}
for i in range(rec.analog_count):
    lbl = rec.analog_channel_ids[i].strip()
    channels[lbl] = np.array (rec.analog[i])


In [ ]:
kVLNbase = 138.0 / math.sqrt(3.0)
MVAbase = 100.0
kAbase = MVAbase / kVLNbase / 3.0

# change signs of P and Q for positive flow out of the plant
def scale_factor(lbl):
  if 'P' in lbl:
    return -1.0 / MVAbase
  elif 'Q' in lbl:
    return -1.0 / MVAbase
  elif 'I' in lbl:
    return 1.0 / kAbase / math.sqrt(2.0)
  elif 'rms' in lbl:
    return 1.0
  elif 'V' in lbl:
    return 1.0 / kVLNbase / math.sqrt(2.0)
  return 1.0

# this is for Plotly color selection; for Matplotlib we'll use default color rotation
class ColorList:
    def __init__(self):
        self.cidx = 0
        self.colors = px.colors.qualitative.Plotly
        self.n = len(self.colors)
    def __call__(self):
        return self.colors[self.cidx]
    def step(self):
        self.cidx = (self.cidx + 1) % self.n
    def setidx(self, cidx):
        self.cidx = cidx
    def getidx(self):
        return self.cidx
    def shiftidx(self, shift):
        self.cidx = (self.cidx + shift) % self.n

In [ ]:
print ('{:s}.cfg/dat contains {:d} points, dt={:.8f}'.format(root, len(t), t[1]-t[0]))
print ('Channel           Min        Max       Mean')
for key, val in channels.items():
    print ('{:10s} {:10.4f} {:10.4f} {:10.4f}'.format (key, np.min(val), np.max(val), np.mean(val)))

Run the following cell to generate the plot using [Plotly](https://plotly.com/python/getting-started/) an interactive plotting library.

In [ ]:
# display all of the channels in groups of three
fig = go.Figure().set_subplots(3, 1, shared_xaxes=True, x_title="Time [s]")
labels = [lbl for lbl in channels]
colors = ColorList()
for i in range(3):
    if i < 2:
        colors.setidx(0)
    for j in range(3):
        label = labels[3*i + j]
        fig.add_trace(go.Scatter(x=t, y=scale_factor(label)*channels[label],
                                 name=label, line_color=colors()),
                                 row=i+1, col=1)
        colors.step()
fig.update_layout(title='Case: ' + root, template="simple_white", width=1200, height=1000)
fig.update_xaxes(showgrid=True, minor_showgrid=True)
fig.update_yaxes(showgrid=True, minor_showgrid=True)
#### Display/Save
## uncomment the line below to save the plot to an html file (still interactive) that can be opened later
# fig.write_html(os.path.join(os.getcwd(), "comtrade_plot_test.html")) 
fig.show()

In [ ]:
# show a zoomed plot of Q for post-processing
fig = go.Figure().set_subplots(1, 1, shared_xaxes=True, x_title="Time [s]")
for lbl in ['A9: Q']:
  fig.add_trace (go.Scatter(x=t, y=scale_factor(lbl) * channels[lbl], name=lbl))
fig.update_xaxes(range=[0.8, 2.0])

fig.update_layout(title='Case: ' + root, template="simple_white", width=1200, height=1000)
fig.update_xaxes(showgrid=True, minor_showgrid=True)
fig.update_yaxes(showgrid=True, minor_showgrid=True)
fig.show()

Run the following cells to generate the plots using [Matplotlib](https://matplotlib.org/)

In [ ]:
# display all of the channels in groups of three
# %matplotlib inline
%matplotlib notebook
# %matplotlib widget
labels = [lbl for lbl in channels]
fig, ax = plt.subplots(3, 1, sharex = 'col', figsize=(10,8), constrained_layout=True)
fig.suptitle ('Case: ' + root)
for i in range(3):
    for j in range(3):
        lbl = labels[3*i + j]
        ax[i].plot(t, scale_factor(lbl) * channels[lbl], label=lbl)
    ax[i].grid()
    ax[i].legend()

plt.show()

In [ ]:
# show a zoomed plot of Q for post-processing
fig, ax = plt.subplots(1, 1, sharex = 'col', figsize=(8,6), constrained_layout=True)
for lbl in ['A9: Q']:
  ax.plot (t, scale_factor(lbl) * channels[lbl], label=lbl, color='green')
ax.set_xlim (0.8, 2.0)
ax.set_ylabel ('perunit')
ax.set_xlabel ('seconds')
ax.legend()
ax.grid()
plt.show()

In [ ]:
# define a signal for post-processing, windowed on the region of interest
t_start = 0.8
t_end = 2.0

#t_start = 0.0
#t_end = 6.0

n1=min(np.argwhere(t>=t_start))[0]
n2=max(np.argwhere(t<=t_end))[0]
lbl = 'A9: Q'
y = scale_factor(lbl)*channels[lbl][n1:n2]
x = t[n1:n2]

# for testing a signal reversed in polarity
#y *= -1.0

# define some key points on the signal
y_init = y[0]
x_init = x[0]
y_final = y[-1]
x_final = x[-1]
y_mean = np.mean(y)
y_max = np.max(y)
x_max = x[np.argmax(y)]
y_min = np.min(y)
x_min = x[np.argmin(y)]
print ('Min   Y={:10.6f} at {:.6f}'.format (y_min, x_min))
print ('Max   Y={:10.6f} at {:.6f}'.format (y_max, x_max))
print ('Mean  Y={:10.6f}'.format (y_mean))
print ('Init  Y={:10.6f} at {:.6f}'.format (y_init, x_init))
print ('Final Y={:10.6f} at {:.6f}'.format (y_final, x_final))

In [ ]:
# look for the 10% and 90% points
pu_reaction = 0.1
pu_rise = 0.9
sb_pu = 0.05

y_reaction = y_init + pu_reaction * (y_final - y_init)
reverse = False
if y_final < y_init:
    reverse = True
    y_peak = y_min
    x_peak = x_min
    idx_reaction = min(np.argwhere(y<=y_reaction))[0]
    y_rise = y_init + pu_rise * (y_min - y_init)
    idx_rise = min(np.argwhere(y<=y_rise))[0]
else:
    y_peak = y_max
    x_peak = x_max
    idx_reaction = min(np.argwhere(y>=y_reaction))[0]
    y_rise = y_init + pu_rise * (y_max - y_init)
    idx_rise = min(np.argwhere(y>=y_rise))[0]
x_reaction = x[idx_reaction]
x_rise = x[idx_rise]
print ('Init  [{:.6f},{:10.6f}]'.format (x_init, y_init))
print ('Start [{:.6f},{:10.6f}]'.format (x_reaction, y_reaction))
print ('Rise  [{:.6f},{:10.6f}]'.format (x_rise, y_rise))
print ('Peak  [{:.6f},{:10.6f}]'.format (x_peak, y_peak))
print ('Final [{:.6f},{:10.6f}]'.format (x_final, y_final))


In [ ]:
# some figures of merit
x_fault = 1.0 # known from the EMT simulation
y_fault = y_init

# wavefront and damping measurements
reaction_time = x_reaction - x_fault
rise_time = x_rise - x_reaction
overshoot = abs(y_peak - y_final) / abs (y_final - y_init)
#TODO: handle the case where there is no overshoot
ln_overshoot = math.log(overshoot)
damping_by_overshoot = -ln_overshoot / math.sqrt (math.pi*math.pi + ln_overshoot*ln_overshoot)

# settling band measurements
sb_pu *= abs(y_final)
sb1 = y_final + sb_pu
sb2 = y_final - sb_pu
nb1 = max(np.argwhere(y>=sb1))[0]
nb2 = max(np.argwhere(y<=sb2))[0]
if nb1 > nb2:
    y_settle = y[nb1]
    x_settle = x[nb1]
else:
    y_settle = y[nb2]
    x_settle = x[nb2]
settling_time = x_settle - x_reaction
# reporting
print ('Reaction Time = {:.6f}'.format (reaction_time))
print ('Rise Time =     {:.6f}'.format (rise_time))
print ('Settling Time = {:.6f}'.format (settling_time))
print ('Overshoot =     {:.6f}'.format (overshoot))
print ('Damping (O) =   {:.6f}'.format (damping_by_overshoot))


In [ ]:
# looking for relative peaks, beginning at the time of the peak
from scipy.signal import find_peaks
sign = 1.0
if reverse:
    sign = -1.0
x_search = x[idx_reaction:]
y_search = y[idx_reaction:]
idx_peaks, _ = find_peaks(sign*y_search, distance=500)
x_peaks = x_search[idx_peaks]
y_peaks = y_search[idx_peaks]

# only use the peaks that keep descending in magnitude
n_peaks = len(y_peaks)
y_dec = []
x_dec = []
last_cycle = 10.0
first_cycle = -1.0
for i in range(len(y_peaks)):
    cycle_peak = abs(y_peaks[i] - y_init)
    if cycle_peak < last_cycle:
        last_cycle = cycle_peak
        y_dec.append(y_peaks[i])
        x_dec.append(x_peaks[i])
        if first_cycle < 0.0:
            first_cycle = cycle_peak
    else:
        break
#print (y_dec)
#print (x_dec)
n_dec = len(y_dec)
if n_dec > 1:
    log_decrement = math.log(first_cycle / last_cycle) / n_dec
    val = 2.0 * math.pi / log_decrement
    damping_by_decrement = 1.0 / math.sqrt(1 + val * val)
    print ('Damping (dec) = {:.6f} over {:d} cycles'.format (damping_by_decrement, n_dec))
else:
    print ('Log decrement not defined for {:d} cycles'.format (n_dec))

In [ ]:
# plot the signal and identified markers
xmark = [x_init, x_fault, x_reaction, x_rise, x_min, x_max, x_settle, x_final]
ymark = [y_init, y_fault, y_reaction, y_rise, y_min, y_max, y_settle, y_final]
xband = [x[0], x[-1]]
yband1 = [sb1, sb1]
yband2 = [sb2, sb2]
fig, ax = plt.subplots(1, 1, sharex = 'col', figsize=(8,6), constrained_layout=True)
ax.plot (x, y, label='signal', color='green')
ax.plot (xband, yband1, label='sb1', linestyle=':', color='magenta')
ax.plot (xband, yband2, label='sb2', linestyle='--', color='magenta')
ax.plot (xmark, ymark, label='markers', marker='o', markersize=6, linestyle='', color='blue')
ax.plot (x_dec, y_dec, label='log dec peaks', marker='o', markersize=4, linestyle='', color='orange')
ax.set_ylabel ('perunit')
ax.set_xlabel ('seconds')
ax.legend()
ax.grid()
plt.show()

In [ ]:
# make an RMSE sample waveform
x_example = [ 0.80,  1.00,  1.05,  1.15,  1.60,  1.80,  2.00]
y_example = [-0.17, -0.17, -1.00, -0.45, -0.55, -0.58, -0.60]
y_base = np.interp (x, x_example, y_example)

# calculate the RMSE, normalized to range of the test
n_base = len(y_base)
y_range = abs (max(y_base) - min(y_base))
diff = (y_base - y) / y_range

rmse = math.sqrt (np.mean(np.square(diff)))
print ('RMSE = {:.4f} pu'.format(rmse))

# show the plot comparing the two 
fig, ax = plt.subplots(1, 1, sharex = 'col', figsize=(8,6), constrained_layout=True)
ax.plot (x, y_base, label='base', color='orange')
ax.plot (x, y, label='signal', color='green')
ax.set_ylabel ('perunit')
ax.set_xlabel ('seconds')
ax.legend()
ax.grid()
plt.show()